# Extract & mangrove priority area data from Dabalà et al.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data extraction

In [3]:
mollweide = '+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs'
proj_eck4 = '+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'
data_dir = Path('../data')

files = ['result_Bio_AllWDPA',
         'result_Bio_WDPA',
         'result_Bio',
         'result_BioServ_10',
         'result_BioServ_100',
         'result_BioServ_1000',
         'result_BioServ_AllWDPA_rmPA',
         'result_BioServ_AllWDPA',
         'result_BioServ_WDPA_rmPA',
         'result_BioServ_WDPA',
         'result_BioServ']

cols = ['ID', 'geometry']
ssps = ['245', '370', '585']

In [3]:
# polygon function
def create_polygon(row):
    points = list(zip(row['x'], row['y']))
    return Polygon(points)

In [ ]:
for filename in files:

    # read file containing geometry
    df = pd.read_csv(data_dir.joinpath(f'{filename}.csv'))

    # collapse geometry columns
    df["geometry_1"] = df["geometry_1"].str.strip("list(c(").astype(float)
    df["geometry_14"] = df["geometry_14"].str.strip("))").astype(float)

    # create x and y columns
    df['x'] = list(zip(df.geometry_1, df.geometry_2, df.geometry_3, df.geometry_4, df.geometry_5, df.geometry_6, df.geometry_7))
    df['y'] = list(zip(df.geometry_8, df.geometry_9, df.geometry_10, df.geometry_11, df.geometry_12, df.geometry_13, df.geometry_14))

    # construct gdf
    # apply function to each row to create a geometry column
    df['geometry'] = df.apply(create_polygon, axis=1)
    # drop unnecessary columns
    df = df[cols]
    # convert df to gdf
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    # set native crs
    gdf.set_crs(mollweide, inplace=True)
    gdf['geometry'] = gdf['geometry'].buffer(0) # buffer by 0 to fix minor topological issues

    # merge with remaining data
    data_df = pd.read_csv(data_dir.joinpath(f'{filename}_no_geom.csv'))
    gdf_merged = gdf.merge(data_df, on='ID')

    # save
    gdf_merged.to_csv(data_dir.joinpath(f'{filename}_gdf.csv'), index=False)

### Join with mangrove data

In [5]:
# load priority area file
cols = ['ID', 'geometry', 'rank', 'AreaGMWKm', 'Fishing_Intensity',
       'soil_carbon', 'biomass_carbon', 'InvFish', 'InvProperty', 'InvPeople',
       'InvCarbon', 'TOT_STOCK', 'POP', 'InvPropertyCost', 'InvPeopleCost',
       'InvFishCost', 'Tot_Carbon', 'InvCarbonCost', 'Fishing_Intensity_prc',
       'TOT_STOCK_prc', 'POP_prc', 'Tot_Carbon_prc', 'country', 'continent']

pa_df = pd.read_csv(data_dir.joinpath('gdf', 'result_BioServ_gdf.csv'))
pa_df = pa_df[cols]
pa_df['geometry'] = gpd.GeoSeries.from_wkt(pa_df['geometry'])
pa_gdf = gpd.GeoDataFrame(pa_df, geometry='geometry', crs=mollweide)

for ssp in ssps:
    df = pd.read_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_ssp{ssp}_2cal_risk.csv'))
    df = df.drop(columns=['index_right'])
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
    gdf = gdf.to_crs(mollweide)
    join = gpd.sjoin_nearest(gdf, pa_gdf, how='inner', max_distance=100)
    join = join.to_crs(epsg=4326)
    join.to_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_ssp{ssp}_2cal_risk_BioServ.csv'), index=False)
